In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
# Load data
file_path = "../data/ChallengeXHEC23022024.xlsx"
excel_data = pd.ExcelFile(file_path)


jan24_df = pd.read_excel(excel_data, sheet_name="JAN24")
clients_df = pd.read_excel(excel_data, sheet_name="clients")
intervenants_df = pd.read_excel(excel_data, sheet_name="intervenants")

In [ ]:
# Analyze start/end times for each type of service
jan24_df["Heure de début"] = pd.to_datetime(
    jan24_df["Heure de début"], format="%H:%M:%S"
).dt.time
jan24_df["Heure de fin"] = pd.to_datetime(
    jan24_df["Heure de fin"], format="%H:%M:%S"
).dt.time
service_constraints = jan24_df.groupby("Prestation").agg(
    {"Heure de début": ["min", "max"], "Heure de fin": ["min", "max"]}
)
service_constraints.columns = [
    "Start Time Earliest",
    "Start Time Latest",
    "End Time Earliest",
    "End Time Latest",
]
service_constraints.reset_index(inplace=True)


print(service_constraints)

In [ ]:
prestation_segmented_data = {prestation: group for prestation, group in jan24_df.groupby('Prestation')}


toilette_data = prestation_segmented_data['TOILETTE']  # Replace 'TOILETTE' with the desired service segmented data


In [ ]:
# Segment clients according to their needs
client_needs = (
    jan24_df.groupby(["ID Client", "Prestation"]).size().unstack(fill_value=0)
)

print(client_needs)

In [ ]:
# Segment days according to the services offered
jan24_df["Day of Week"] = jan24_df["Date"].dt.dayofweek
jan24_df["Day Type"] = jan24_df["Day of Week"].apply(
    lambda x: "Weekend" if x > 4 else "Weekday"
)
day_segmentation = (
    jan24_df.groupby(["Day Type", "Prestation"]).size().unstack(fill_value=0)
)

day_segmentation.T

In [ ]:
day_seg_inv = day_segmentation.T
print(
    "Weekday services: ", day_seg_inv[day_seg_inv.Weekday > 0].index.to_list()
)
print(
    "Weekend services: ", day_seg_inv[day_seg_inv.Weekend > 0].index.to_list()
)

# Plotting agenda of Caretakers

In [ ]:
jan24_df["Start DateTime"] = pd.to_datetime(
    jan24_df["Date"].astype(str) + " " + jan24_df["Heure de début"].astype(str)
)
jan24_df["End DateTime"] = pd.to_datetime(
    jan24_df["Date"].astype(str) + " " + jan24_df["Heure de fin"].astype(str)
)

In [ ]:
# TODO: add commute time & waittime
# TODO: add client id in hover description

In [ ]:
def plot_agenda_for_intervenant(intervenant_id):
    intervenant_agenda = jan24_df[jan24_df["ID Intervenant"] == intervenant_id]
    intervenant_agenda_sorted = intervenant_agenda.sort_values(
        by=["Date", "Heure de début"]
    )

    df_timeline = intervenant_agenda_sorted.copy()
    df_timeline["Start"] = pd.to_datetime(df_timeline["Start DateTime"])
    df_timeline["Finish"] = pd.to_datetime(df_timeline["End DateTime"])
    df_timeline["Task"] = df_timeline["Prestation"]
    df_timeline["Resource"] = df_timeline["ID Intervenant"].astype(str)

    fig = px.timeline(
        df_timeline,
        x_start="Start",
        x_end="Finish",
        y="Task",
        color="Resource",
    )
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(title=f"Agenda for Intervenant ID: {intervenant_id}")

    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list(
                    [
                        dict(
                            count=1,
                            label="1D",
                            step="day",
                            stepmode="backward",
                        ),
                        dict(
                            count=7,
                            label="1W",
                            step="day",
                            stepmode="backward",
                        ),
                        dict(step="all"),
                    ]
                )
            ),
            rangeslider=dict(visible=True),
            type="date",
        )
    )

    fig.show()

In [ ]:
for intervenant_id in jan24_df["ID Intervenant"].unique():
    plot_agenda_for_intervenant(intervenant_id)